In [12]:
#pip install pandas azure-storage-blob azure-functions python-dotenv langchain-openai langchain pinecone-client pyarrow adlfs fsspec langchain_pinecone -q

In [13]:
#pip install azure-storage-file-datalake -Q

In [3]:
pip freeze ->requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [8]:
import time
import pandas as pd
from datetime import datetime,timezone
from external.blob_storage import BlobStorage
from app.repositories.blob_storage_reader import BlobStorageReader
from dotenv import load_dotenv
from datetime import datetime,timedelta,timezone
load_dotenv()

True

# test caso de uso landing to raw

In [17]:
from external.data_lake_pandas import DataLakePandas
from app.repositories.pandas_loader import DataLakeLoader
from external.blob_storage import BlobStorage
from app.repositories.blob_storage_reader import BlobStorageReader
from app.use_cases.data_preprocessing_raw import DataPreprocessingRaw #
from app.use_cases.data_prep_delta_raw import DataPreprocessingDeltaRaw #
from app.repositories.pandas_reader import DataLakeReader
from external.data_lake_storage import DataLakeStorage
from dotenv import load_dotenv
load_dotenv()

True

full

In [10]:
sink_container_name = "raw"
filename = 'youtube_data/youtube_data'
source_container_name = "landing"
blob_storage = BlobStorage(source_container_name)
bs_reader = BlobStorageReader(blob_storage)
dl_storage = DataLakePandas()
dl_loader = DataLakeLoader(dl_storage)
use_case = DataPreprocessingRaw(bs_reader, dl_loader)  
use_case.execute("source_blob", destination=sink_container_name,filename=filename)

abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2024
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2023
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2022
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2021
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2020
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2019
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2014


delta

In [25]:
container_name = "landing"
blob_storage = BlobStorage(container_name)
bs_reader = BlobStorageReader(blob_storage)
dl_storage = DataLakeStorage("raw")
dl_pandas = DataLakePandas()
dl_loader = DataLakeLoader(dl_pandas)
dl_reader = DataLakeReader(dl_pandas,dl_storage)


In [26]:
from datetime import datetime,timedelta,timezone

today= datetime.now(tz=timezone.utc)
print(f"hoy {today}")
delta_day = timedelta(days=1)
delta_hour = timedelta(hours=today.hour)+timedelta(minutes=today.minute)+timedelta(seconds=today.second)
delta_date = today-delta_day-delta_hour
delta_date

hoy 2024-04-23 15:14:13.807857+00:00


datetime.datetime(2024, 4, 22, 0, 0, 0, 807857, tzinfo=datetime.timezone.utc)

In [27]:
from app.use_cases.data_prep_delta_raw import DataPreprocessingDeltaRaw
from datetime import datetime,timedelta,timezone
today= datetime.now(tz=timezone.utc)
print(f"hoy {today}")
container_sink = "raw"
filename= "youtube_data/youtube_data"

hoy 2024-04-23 15:14:16.287018+00:00


In [28]:
use_case_delta_raw = DataPreprocessingDeltaRaw(bs_reader, dl_loader,dl_reader)  
use_case_delta_raw.execute("source_blob", 
                           destination=container_sink,
                           filename=filename,
                           delta_hour=today.hour,
                           delta_day=1)

abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2024


# test caso de uso raw to curated

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
connection_string = os.environ.get('DATALAKE_CONNECTION_STRING')

In [29]:
from datetime import datetime,timedelta,timezone

today= datetime.now()
print(f"hoy {today}")
delta_day = timedelta(days=15)
delta_hour = timedelta(hours=today.hour)+timedelta(minutes=today.minute)+timedelta(seconds=today.second)
delta_date = today-delta_day-delta_hour
delta_date

hoy 2024-04-23 12:52:44.733768


datetime.datetime(2024, 4, 8, 0, 0, 0, 733768)

In [1]:
from app.use_cases.data_preprocessing_curated import DataPreprocessingCurated 
from external.data_lake_pandas import DataLakePandas
from external.data_lake_storage import DataLakeStorage
from app.repositories.pandas_loader import DataLakeLoader
from app.repositories.pandas_reader import DataLakeReader


In [2]:
dl_pandas = DataLakePandas()
dl_storage = DataLakeStorage("raw")
dl_reader = DataLakeReader(dl_pandas,dl_storage)
dl_loader = DataLakeLoader(dl_pandas)
use_case_curated = DataPreprocessingCurated(dl_reader,dl_loader)

carga full

In [32]:
container_source = "raw"
container_sink = "curated"
filename= "youtube_data"
filter = None
use_case_curated.execute(container_source, container_sink,filename, filter)

youtube_data/youtube_data_2014
youtube_data/youtube_data_2019
youtube_data/youtube_data_2020
youtube_data/youtube_data_2021
youtube_data/youtube_data_2022
youtube_data/youtube_data_2023
youtube_data/youtube_data_2024
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data_2014
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data_2019
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data_2020
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data_2021
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data_2022
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data_2023
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data_2024


carga delta


In [3]:
from datetime import datetime
from app.use_cases.data_prep_delta_curated import DataPreprocessingDeltaCurated #

use_case_delta_curated = DataPreprocessingDeltaCurated(dl_reader,dl_loader)

In [4]:
filter_date = datetime.now().strftime('%Y-%m-%d')
filter_date

'2024-04-23'

In [6]:
container_source = "raw"
container_sink = "curated"
filename= "youtube_data"
filter = None
#filter = [("publish_date", ">=", filter_date)]
use_case_delta_curated.execute(container_source, container_sink,filename,1,0, filter)

hoy 2024-04-23 13:08:06.986944
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data_2014
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data_2019
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data_2020
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data_2021
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data_2022
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data_2023
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data_2024


etl full

In [1]:
import azure.functions as func
import logging
from datetime import datetime
import os
import io

from external.blob_storage import BlobStorage
from external.data_lake_pandas import DataLakePandas
from external.data_lake_storage import DataLakeStorage

from app.repositories.pandas_loader import DataLakeLoader
from app.repositories.blob_storage_reader import BlobStorageReader
from app.repositories.pandas_reader import DataLakeReader

from app.use_cases.data_preprocessing_curated import DataPreprocessingCurated
from app.use_cases.data_preprocessing_raw import DataPreprocessingRaw 

from dotenv import load_dotenv
load_dotenv()


import azure.functions as func
import logging

bp2 = func.Blueprint()

container_name = "landing"
blob_storage = BlobStorage(container_name)
bs_reader = BlobStorageReader(blob_storage)
dl_pandas = DataLakePandas()
dl_loader = DataLakeLoader(dl_pandas)

dl_storage = DataLakeStorage("raw")
dl_reader = DataLakeReader(dl_pandas,dl_storage)

use_case_full_raw = DataPreprocessingRaw(bs_reader, dl_loader)  

use_case_full_curated = DataPreprocessingCurated(dl_reader, dl_loader)  


try:
    timestamp = datetime.now()
    filter_date = timestamp.strftime('%Y-%m-%d')
    
    container_source = "landing"
    container_sink = "raw"
    filename= "youtube_data"
    use_case_full_raw.execute(container_source, 
                            container_sink,
                            filename
                            )
    logging.info('external to raw')
    container_source = "raw"
    container_sink = "curated"
    filename= "youtube_data"
    filter = None
    #filter = [("publish_date", ">=", filter_date)]
    use_case_full_curated.execute(container_source, container_sink,filename, filter)
    logging.info('raw to curated')
    msn= 'etl full load function ran at %s'
except Exception as e:
    msn = 'Error ejecutando la etl at %s'
    logging.info(msn)
    print(msn)



leer datos
procesar data
cargar data
[2024 2023 2022 2021 2020 2019 2014]
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2024
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2023
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2022
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2021
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2020
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2019
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data_2014
youtube_data/youtube_data_2014
youtube_data/youtube_data_2019
youtube_data/youtube_data_2020
youtube_data/youtube_data_2021
youtube_data/youtube_data_2022
youtube_data/youtube_data_2023
youtube_data/youtube_data_2024
[2014 2019 2020 2021 2022 2023 2024]
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data_2014
abfs://sadat

: 

### etl delta

In [9]:
import azure.functions as func
import logging
from datetime import datetime
import os

from external.blob_storage import BlobStorage
from external.data_lake_pandas import DataLakePandas
from external.data_lake_storage import DataLakeStorage

from app.repositories.pandas_loader import DataLakeLoader
from app.repositories.blob_storage_reader import BlobStorageReader
from app.repositories.pandas_reader import DataLakeReader

from app.use_cases.data_prep_delta_curated import DataPreprocessingDeltaCurated
from app.use_cases.data_prep_delta_raw import DataPreprocessingDeltaRaw 

from dotenv import load_dotenv
load_dotenv()


DELTA_DAYS = os.environ.get('DELTA_DAYS')
DELTA_HOURS = os.environ.get('DELTA_HOURS')
SCHEDULE = os.environ.get('CRON_SCHEDULE')

bp1 = func.Blueprint()
container_name = "landing"
blob_storage = BlobStorage(container_name)
bs_reader = BlobStorageReader(blob_storage)
dl_pandas = DataLakePandas()
dl_loader = DataLakeLoader(dl_pandas)

dl_storage = DataLakeStorage("raw")
dl_reader = DataLakeReader(dl_pandas,dl_storage)

use_case_delta_raw = DataPreprocessingDeltaRaw(bs_reader, dl_loader, dl_reader)  

use_case_delta_curated = DataPreprocessingDeltaCurated(dl_reader, dl_loader)  


timestamp = datetime.now()
filter_date = timestamp.strftime('%Y-%m-%d')
try:
    container_source = "landing"
    container_sink = "raw"
    filename= "youtube_data/youtube_data"
    filter = None
    delta_day= int(DELTA_DAYS)
    delta_hour= int(DELTA_HOURS)
    use_case_delta_raw.execute(container_source, 
                            container_sink,
                            filename,
                            delta_day=delta_day,
                            delta_hour= delta_hour
                            )
    logging.info('external to raw')
    print('external to raw')
    container_source = "raw"
    container_sink = "curated"
    filename= "youtube_data"
    filter = None
    #filter = [("publish_date", ">=", filter_date)]
    use_case_delta_curated.execute(container_source, container_sink,filename,delta_day,delta_hour, filter)
    logging.info('raw to curated')
    print('raw to curated')
except Exception as e:
    msn = f'Error {e} , ejecutando la etl'
    logging.info(msn)
    print(msn)


[2024]
abfs://sadatalakeproyecto.dfs.core.windows.net/raw/youtube_data/youtube_data/youtube_data/youtube_data_2024
external to raw
hoy 2024-04-24 14:59:06.205497
[2024 2014 2019 2020 2021 2022 2023]
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data/youtube_data/youtube_data_2024
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data/youtube_data/youtube_data_2014
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data/youtube_data/youtube_data_2019
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data/youtube_data/youtube_data_2020
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data/youtube_data/youtube_data_2021
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data/youtube_data/youtube_data_2022
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/youtube_data/youtube_data/youtube_data/youtube_data_2023
raw to curated


## embedding

In [173]:
import os
EMBEDDING_DEPLOYMENT = os.environ["AZURE_EMBEDDING_DEPLOYMENT"] 
index_name = "video-transcription-index"

In [174]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
#from pinecone import Pinecone
#from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore

In [175]:
embedding = AzureOpenAIEmbeddings(azure_deployment=EMBEDDING_DEPLOYMENT)
vectordb = PineconeVectorStore(index_name=index_name,embedding=embedding)

In [179]:
retriever = vectordb.as_retriever(search_type="mmr")
matched_docs = retriever.get_relevant_documents(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

bitcoin: listo para comprar? | btc | cryptos | etf | eth. sigo comprando sigo comprando sigo comprando yo sigo comprando pa pa pa pa oh ya estamos en vivo hola mis amigos cómo les va les saluda gre quiroga noticias noticias ojo 320 millones de dólares liquidados entre ayer y hoy de los cuales raramente casi 300 millones en long por qué no sé eh otra noticia muy importante sigue vendiendo grayscale otra noticia muy importante sigue habiendo compras otra noticia importante rara rara noticia pero veo que los países se están armando de bitcoin ayer hablaba alguien me informaba de que estados unidos compró unos bitcoins en enero olvidemos el día pero compró como 170,000 o le aparecieron no bueno a china también le aparecieron ahora yo no sé si los tenía antes pero en un update de los tesoros de bitcoin le aparecieron a china 250,000 bitcoins creo que sí y a venezuela le aparecieron otro poquito y a bhután le aparecieron otro poquito de bitcoins los países están acumulando bi

In [181]:
from external.lang_chain_manager import LangChainManager

In [182]:
chain_manager = LangChainManager()

In [189]:
index_name = "video-transcription-index"
query = "esta bien comprar bitcoin ?"
today = datetime.now().strftime('%Y-%m-%d')
today

'2024-04-16'

In [190]:
chain_manager.document_search(index_name,query,metadata_filter= { "publish_date": '2024-04-15' })

[Document(page_content='iodía compré como 120 más yo voy dándole yo voy dándole poco a poco mi plan cuál es vean mi plan vean cómo yo clarifico mi mi mis metas mis metas es antes de que termine la onda 4 tener entre medio y un bitcoin más entre medio y un bitcoin más antes de que termine la onda 4 o cuando termine verdad porque yo en onda 5 ya no compro nada más y ya llevo según lo que he visto en este en estos exchange donde acumulo que ustedes lo pueden chequear todos los días eh ya llevo como 008 por ahí más o menos ya estoy completando 010 y apenas está empezando este retroceso o rango verdad entonces las cosas van como bien eh daniel jaramillo dice nos vamos a cero coquetín dice esto es eh momentáneo eh césar sánchez dice se viene otra impresión de dólares a lo bruto edwin web dice profe israel ya confirmó eh contraatacar el contragolpe dijeron que estaban preparados y parece que es inevitable yo le dije que era escorpión deco martínez dice momento de incertidumbre con este confli